In [ ]:
import argparse
import pandas as pd
import numpy as np
import os


class DataProcessor:
    def __init__(self):
        pass

    def load_data(self, raw_data_filename):
        df = pd.read_csv(raw_data_filename)
        return df

    def sort_by_datetime(self, df):
        return df.sort_values(by=["Datetime"])

    def extract_date_features(self, df):
        df["DayOfWeek"] = pd.to_datetime(df["Datetime"]).dt.dayofweek
        df["Hour"] = pd.to_datetime(df["Datetime"]).dt.hour
        df["Minute"] = pd.to_datetime(df["Datetime"]).dt.minute
        return df

    def one_hot_encode_day_of_week(self, df, max_day_of_week=4):
        for i in range(max_day_of_week + 1):
            df[f"DayOfWeek_{i}"] = (df["DayOfWeek"] == i).astype(int)
        df.drop(columns=["DayOfWeek"], inplace=True)
        return df

    def prepare_data(self, df, lag):
        df = df.copy()

        # Prepare lagged features
        lagged_data = {}
        for i in range(1, lag + 1):
            for col in ["Open", "Close", "High", "Low", "Volume"]:
                lagged_data[f"{col}_lag_{i}"] = df[col].shift(i)

        lagged_df = pd.DataFrame(lagged_data)
        df = pd.concat([df, lagged_df], axis=1)

        # Create target variable: whether Close price will be higher in 3 minutes
        df["Close_target"] = (df["Close"].shift(-3) > df["Close"]).astype(int)

        df.dropna(inplace=True)
        return df


def parse_args():
    parser = argparse.ArgumentParser()
    parser.add_argument('--raw_data_filename', type=str, required=True, help='Filename of raw training data.')
    parser.add_argument('--output_path', type=str, required=True, help='Directory where processed data will be saved.')
    return parser.parse_args()


if __name__ == "__main__":
    args = parse_args()

    processor = DataProcessor()

    # Load raw training data
    df = processor.load_data(args.raw_data_filename)

    # Process training data
    df = processor.sort_by_datetime(df)
    df = processor.extract_date_features(df)
    df = processor.one_hot_encode_day_of_week(df)

    lag = 30
    df = processor.prepare_data(df, lag)

    # Save processed training data to CSV
    os.makedirs(args.output_path, exist_ok=True)
    output_filename = os.path.join(args.output_path, "train.csv")
    df.to_csv(output_filename, index=False)

    print(f"Processed training data saved to {output_filename}")
